# 네이버 블로그로 파급력을 측정합니다.

In [ ]:
import urllib.request
import json
import pandas as pd
import re
# 글에서 해당 키워드 찾기
from konlpy.tag import Kkma
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# 트리맵
import squarify

<naverapi 응답 구조>  
lastBuildDate : api 요청한 시각  
total : 관련 블로그 수  
start : 시작  
display : 가져온 갯수  
items : 블로그  
<items 구조>  
title : 블로그 제목    
link : 블로그 링크  
description : 블로그 내용  
bloggername : 블로그 이름  
bloggerlink : 블로그 메인화면  
postdate : 게시날짜  

가정1 : 이영자님의 영향을 받은 사람은 32명이다.  
가정2 : 이영자님의 영향을 받은 사람은 144명이다.

# 크롤링

## 1. 주요 클래스

In [ ]:
class NaverData:
    count = -1
    power = pd.DataFrame(columns = ["id","resource", "restaurant", "oldestDate", "reference_before", "referenceDate",
                                    "reference_after", "mostRecentDate", "blogTotal", "blogDisplay", "resource_count"])
    power.set_index(['id'], inplace=True)
    findData = pd.DataFrame(columns = ["id", "resource", "restaurant", "data"])
    findData.set_index(['id'], inplace=True)
    
    def __init__(self, resource, restaurant, display=200, referenceDate=None, data=None):
        self.resource = resource
        self.restaurant = restaurant
        if data is None:
            # 네이버 api로 데이터 가져오기
            naverData = NaverData.naverBlog(restaurant, display)
            self.lastBuildDate = naverData["lastBuildDate"]
            self.total = naverData["total"]
            self.display = display
            self.items = NaverData.cleanIteams(pd.DataFrame(naverData["items"]))
        else:
            # 로컬 저장소에 있는 데이터 가져오기
            tempData = data
            self.lastBuildDate = tempData["lastBuildDate"]
            self.total = tempData["total"]
            self.display = tempData["display"]
            self.items = tempData["items"]
        self.oldestDate = self.findOldestDate()
        self.mostRecentDate = self.findMostRecentDate()
        self.referenceData = self.findResource()
        self.referenceDate = referenceDate
        NaverData.count = NaverData.count + 1
        self.id = NaverData.count
        self.addPower()
        NaverData.findData.loc[self.id] = [resource, restaurant, self]
        print(resource, restaurant)
    
                                               
    def __del__(self):
        NaverData.power.drop(self.id, inplace=True)
        NaverData.findData.drop(self.id, inplace=True)
        print(self.id, "번 데이터를 지웁니다.")
    
    # 네이버 ReatAPI
    def naverBlog(query, num) :
        client_key = 'izGsqP2exeThwwEUVU3x'
        client_secret = 'WrwbQ1l6ZI'
        encText = urllib.parse.quote_plus(query)
        count = [[i,100] for i in range(1, num, 100)]
        if num % 100:
            count[-1][1] = num % 100 
        for i in count:
            naver_url = 'https://openapi.naver.com/v1/search/blog.json?query=' + encText + '&display=' + str(i[1]) +'&start=' + str(i[0])
            # request 객체에 add하기
            request = urllib.request.Request(naver_url)
            request.add_header("X-Naver-Client-Id",client_key)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            # 응답받은 코드가 정상적인지 확인하기
            rescode = response.getcode()
            if(rescode == 200):
                response_body = response.read()
                if i[0] == 1:
                    dataList = json.loads(response_body)
                else:
                    dataList["items"] = dataList["items"] + json.loads(response_body)["items"]
            else:
                print('오류 코드 : ' + rescode)
        return dataList
    
    def cleanIteams(itemsdf):
        """
        items데이터프라임 -> items데이터프라임
         - items데이터프라임 title열과 description열에서 <b></b>&quot; 등 없얘기
         - items데이터프레임 postdate 타입을 datetime으로 변경하기
        """
        for i in range(itemsdf.shape[0]):
            # title열 데이터 정리하기
            title = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"title"]))
            title = re.sub("&.{1,5};", "", title)
            title = re.sub(" ( )+", "", title)
            itemsdf.loc[i,"title"] = title
            # description열 데이터 정리하기
            description = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"description"]))
            description = re.sub("&.{1,5};", "", description)
            description = re.sub(" ( )+", "", description)
            itemsdf.loc[i,"description"] = description
            # postdate열 타입을 datetime로 바꾸기
            itemsdf['postdate'] = pd.to_datetime(itemsdf['postdate'])
        return itemsdf
    
    def findResource(self):
        """
        items 데이터프레임 title열과 description열에서 resource단어가 들어간 데이터를 찾아 줍니다.
        """
        kkma = Kkma()
        mask = (self.items["title"] + self.items["description"]).map(kkma.nouns).map(lambda x: self.resource in x)
        return self.items.loc[mask]
    
    def findReferenceDate(self):
        """
        찾아온 데이터 중 블로그 제목과 설명에 제공자이름이 들어간 데이터 중 가장 오래된 데이터를 찾습니다.
        """
        if len(self.referenceData) <= 0:
            print("해당 게시물이 없습니다.")
            return pd.to_datetime('2099-12-31')  
        else:
            return self.referenceData.sort_values(by='postdate', ascending=True).iloc[0]["postdate"]
        
    def findOldestDate(self):
        """
        찾아온 데이터 중 가장 오래된 데이터를 찾습니다.
        """
        return self.items.sort_values(by='postdate', ascending=True).iloc[0]["postdate"]
    
    def findMostRecentDate(self):
        """
        찾아온 데이터 중 가장 최근 데이터를 찾습니다.
        """
        return self.items.sort_values(by='postdate', ascending=False).iloc[0]["postdate"]
    
    def countBF(self):
        """
        기준일을 기준으로 이전 게시물 갯수와 이후 게시물 갯수를 세줍니다.
        반환값 : {'before':X, 'after': X, 'resource_after':X}
        """
        # 기준일을 받지 않으면 게시물 중 가장 오래된 날짜로 지정한다.
        if self.referenceDate is None:
            self.referenceDate = self.findReferenceDate()
        before = len(self.items.loc[self.items['postdate'] < self.referenceDate])
        after = len(self.items.loc[self.items['postdate'] >= self.referenceDate])
        resource_after = len(self.referenceData.loc[self.referenceData['postdate'] >= self.referenceDate])
        return {'before':before, 'after': after, 'resource_after': resource_after}
    
    def addPower(self):
        bf = self.countBF()
        NaverData.power.loc[self.id] = [self.resource, self.restaurant, self.oldestDate, bf["before"], self.referenceDate,
                                        bf["after"], self.mostRecentDate, self.total, self.display, bf['resource_after']]

## 2-1. 데이터 수집 (주의! 오래걸립니다..)로컬 저장소에 저장된 데이터 가져오기

In [ ]:
# rowdata 클래스로 만들기
temp = pd.read_csv("./data/naver_blog/total_power.csv")
for row in range(len(temp)):
    file = "./data/naver_blog/rowdata/" + temp.iloc[row]["restaurant"] + "_" + temp.iloc[row]["resource"] + ".csv"
    temp_items = pd.read_csv(file)
    data = {"lastBuildDate": None, "total": temp.iloc[row]["blogTotal"], "display" : len(temp_items),
           "items" : temp_items}
    try:
        NaverData(temp.iloc[row]["resource"], temp.iloc[row]["restaurant"], data=data)
    except:
        pass

In [ ]:
# 전체 정보 가져오기
naver_power = pd.read_csv("./data/naver_blog/total_power.csv")

## 2-2. 데이터 수집(naver Rest API)

In [ ]:
# 음식점 데이터(분석대상 데이터) 가져오기
df = pd.read_excel("./output/음식점 모음.xlsx", sheet_name="Sheet1")
resource_rastaurantDF = pd.DataFrame(columns = ["resource","rastaurant"])
# 데이터 변환하기
for col in df.columns:
    rastaurant = df[col].dropna()
    temp = pd.DataFrame({"rastaurant": rastaurant, "resource" : col})
    resource_rastaurantDF = resource_rastaurantDF.append(temp)
resource_rastaurantDF.reset_index(inplace=True)
resource_rastaurantDF.drop("index", axis=1, inplace=True)

In [ ]:
# naver API
# 만약 멈췄을때를 대비해서 미리미리 저장합니다
# 식당데이터를 20개씩 자르기
count = list(range(0, len(resource_rastaurantDF), 20)) + [len(resource_rastaurantDF)]
for i in range(len(count)-1):
    for row in range(count[i], count[i+1]):
        # 네이버 api로 데이터 요청하기
        tempNaverData = NaverData(resource_rastaurantDF.loc[row, "resource"], resource_rastaurantDF.loc[row, "rastaurant"])
        # 블로그 데이터 저장하기
        file = "./data/naver_blog/rowdata/" + NaverData.findData.loc[row, "restaurant"] + "_" + NaverData.findData.loc[row, "resource"] + ".csv"
        NaverData.findData.loc[row, "data"].items.to_csv(file, index=False)

## 3. 파급력 측정하기
네이버 데이터 : 음식점과 관련성이 높은 블로그글 순으로 200개의 데이터를 가져옵니다.  
기준일 : 네이버 데이터 중 제공기관 키워드가 블로그 제목이나 간략한 설명에 들어간 글 중 가장 오래전에 포스트된 날짜  
        (관련된 글이 없다면 기준일은 "2099-12-31 00:00:00"이 됩니다.)  
        
### 가게에 대한 신뢰도 : 미디어가 해당 음식점의 영향력에 얼마나 주었나
power3 : 해당 미디어의 영향력  
reliability2 : 영향력이 얼마나 큰가
### 파급력의 신뢰도 : 미디어 파급력에 영향을 받아서 글을 작성할 확률
reliability

#### (후보) 미디어의 파급력 : 미디어가 해당 음식점의 영향력에 얼마나 주었나
측정 : 기준일을 기준으로 전과 후에 얼마나 급격하게 포스팅 갯수가 늘어났나  

power = (reference_after - reference_before) / 200?  
power2 = [resource_count/ {(reference after - resource_count)/ reference before}+ reference before] / reference before    
power3 = (기준일 후 게시물 개수 / [(가장 마지막 게시물 날짜- 기준일) / (마지막날짜 - 오래된 날짜) * 100] - 기준일 전 게시물 개수 / [(기준일 - 게시물 중 가장 오래된 날짜) / (마지막날짜 - 오래된 날짜) * 100]


(power값이 높을수록 파급력이 좋다. 최댓값 : 200, 최솟값 : -200)  
문제점 : 해당 미디어키워드는 고려하지 않았으므로 미디어에 영향을 받아서 간건지 아닌지 알 수 없음
#### (후보) 미디어의 파급력의 신뢰성 : 측정한 파급력을 얼마나 신뢰해야 할것인가
측정 : 기준일을 기점으로 이후 포스팅되는 글 중 해당 미디어를 키워드를 작성한 글의 퍼센트  
reliability = resource_count / reference_after * 100%  
reliability2 = reference_after/(reference_affter - resource_count) *100%  
(reliability값이 높을수록 측정한 파급력을 더 믿을 수 있다.)
문제점 : 해당 미디어에 영향을 받았음에도 블로그 제목이나 간략한 내용에 작성하지 않은 사람이 있을 수 있다. 따라서 신뢰성은 매우 낮게 나올 수 밖에 없다.)

#### 가정
	restaurant	reference_before	referenceDate	reference_after	blogTotal	blogDisplay	resource_count
	산골농원	56	2019-05-24 00:00:00	144	5101	200	32
가정1 : 이영자님의 영향을 받은 사람은 32명이다.  
가정2 : 이영자님의 영향을 받은 사람은 144명이다.

In [ ]:
# 파급력 열 추가히기
# 파일 가져오기
naver_total = pd.read_csv("./data/naver_blog/total_power.csv")
# 형 변환
naver_total["oldestDate"] = pd.to_datetime(naver_total["oldestDate"]).apply(pd.Timestamp)
naver_total["referenceDate"] = pd.to_datetime(naver_total["referenceDate"]).apply(pd.Timestamp)
naver_total["mostRecentDate"] = pd.to_datetime(naver_total["mostRecentDate"]).apply(pd.Timestamp)
# power3열 추가
# (기준일 후 게시물 개수 / [(가장 마지막 게시물 날짜- 기준일) / (마지막날짜 - 오래된 날짜) * 100] - 기준일 전 게시물 개수 / [(기준일 - 게시물 중 가장 오래된 날짜) / (마지막날짜 - 오래된 날짜) * 100]
# [기준일 후 게시물 개수 / (가장 마지막 게시물 날짜- 기준일) - 기준일 전 게시물 개수 / (기준일 - 게시물 중 가장 오래된 날짜)]* 100 *(마지막날짜 - 오래된 날짜)
naver_total["power3"] = ( naver_total["reference_after"] / pd.to_numeric((naver_total["mostRecentDate"] - naver_total["referenceDate"]).dt.days, downcast='integer') - naver_total["reference_before"] / pd.to_numeric((naver_total["referenceDate"] - naver_total["oldestDate"]).dt.days, downcast='integer')) * pd.to_numeric((naver_total["mostRecentDate"] - naver_total["oldestDate"]).dt.days, downcast='integer') / 100
# reliability2열 추가
naver_total["reliability2"] = naver_total["resource_count"] / (naver_total["reference_after"] - naver_total["resource_count"]) * 100
# reliability열 추가
naver_total["reliability"] = naver_total["resource_count"] / naver_total["reference_after"] * 100
# 결측치 처리 
naver_total['power3'].fillna(0, inplace=True)
naver_total['reliability2'].fillna(100, inplace=True)
naver_total['reliability'].fillna(0, inplace=True)
# 그룹열 만들기
연예인 = ["이영자", "백종원", "성시경", "신동엽", "최자", "비(정지훈)", "문세윤", "김준현"]
유튜버 = ["더들리", "츄릅켠"]
방송프로그램 = ["수요미식회", "생생정보통", "전참시", "생활의 달인", "6시 내 고향", "골목식당", "생방송 오늘 저녁", "모닝 와이드", "맛있는 녀석들", "생방송 투데이"]
groups = [연예인, 유튜버, 방송프로그램]
groups_names = ["연예인", "유튜버", "방송프로그램"]
for i, groups in enumerate(groups):
    for media in groups:
        naver_total.loc[naver_total["resource"] == media, "group"] = groups_names[i]
display(naver_total)
naver_total.to_csv("./data/naver_blog/total_power.csv", index=False)

## 4. 확인하기

In [ ]:
display(NaverData.power)

## 5. 저장하기

In [ ]:
# 파급력 저장하기
NaverData.power.to_csv("./data/naver_blog/total_power.csv", index=False)

In [ ]:
# 각각의 items데이터 저장하기
for row in range(len(NaverData.findData)):
    file = "./data/naver_blog/rowdata/" + NaverData.findData.loc[row, "restaurant"] + "_" + NaverData.findData.loc[row, "resource"] + ".csv"
    NaverData.findData.loc[row, "data"].items.to_csv(file, index=False)

In [ ]:
# 각각의 rowdata_resource 데이터 저장하기
for row in range(len(NaverData.findData)):
    file = "./data/naver_blog/rowdata_resource/" + NaverData.findData.loc[row, "restaurant"] + "_" + NaverData.findData.loc[row, "resource"] + ".csv"
    NaverData.findData.loc[row, "data"].referenceData.to_csv(file, index=False)

# 해볼 수 있는일

In [ ]:
# 클래스 메소드 만들기
mask = (NaverData.findData["resource"] == "김준현") & (NaverData.findData["restaurant"] == "원조수구레")
NaverData.findData.loc[mask].iloc[0]["data"].items

# 실험

In [ ]:
# html 빼기
text = "민락<동 얼큰칼국수&quot;<b>츄릅켠</b> 님 방송 보고 다녀옴!!!&quot;"
text = re.sub("<b>|</b>", " ", text)
text = re.sub("&.{1,5};", "", text)
print(text)

In [ ]:
sample = "해운대 원조할매국밥을 찾아가듯이 이 분위기와 쌈밥이 땡길때면 재방문할 의향이 있네요..."
kkma = Kkma() 
kkma.nouns(sample)
# pprint(kkma.morphs(sample))
# pprint(kkma.pos(sample))

In [ ]:
naverdata1 = NaverData("츄릅켠", "소문난원조쌈밥")
naverdata2 = NaverData("김준현", "원조수구레")

In [ ]:
plt.plot([10, 20, 30, 40], [1, 4, 9, 16], c="b",
         lw=5, ls="--", marker="o", ms=15, mec="g", mew=5, mfc="r")
plt.title("스타일 적용 예")
plt.show()

# 문제모음

1. 인스턴스가 삭제가 안됩니다.
2. 파일 데이터를 불러올때 몇몇의 파일을 읽어오지 못합니다.
3. seaborn에서 한글 폰트를 인식하지 못합니다.

In [ ]:
# 폰트설정2
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

# 폰트설정3
jupyter notebook 내 그래프를 바로 그리기 위한 설정
%matplotlib inline
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

# 폰트설정4
%matplotlib inline
import platform
path = './data/THEdog.ttf'
from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unkonwn system... sorry~~~~')